In [1]:
import pandas as pd
import numpy as np
from calculator import *
from numpy import random
from math import pi
import turtle as tt
import matplotlib.cm as cm
from tqdm import tqdm
import matplotlib.pyplot as plt
import statsmodels.api
import pandas as pd
import pingouin as pg
from statsmodels.tsa.stattools import *
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
from sklearn import linear_model

In [2]:
arsenic = pd.read_csv("data/arsenic.csv")
copper = pd.read_csv("data/copper.csv")
sulfate = pd.read_csv("data/sulfate.csv")
boran = pd.read_csv("data/boran.csv")
chloride = pd.read_csv("data/chloride.csv")
molybdenum = pd.read_csv("data/molybdenum.csv")
vanadium = pd.read_csv("data/vanadium.csv")
# uranium = pd.read_csv("data/uranium.csv")

In [3]:
vanadium.describe()

,WellDepth,Time,SampleNumber,EntityId,LabId,ReliabilityId,LatitudeDD,LongitudeDD,ParameterCode,ParameterValue,PlusMinus
count,2200.000000,2243.000000,2243.000000,2243.000000,2243.000000,2241.000000,2243.000000,2243.000000,2243.0,2243.000000,0.0
mean,1063.632727,1213.478377,1.000446,1.879180,21.461436,7.023650,30.499593,-96.998081,1085.0,3.124730,NaN
std,917.216781,259.505701,0.021115,3.080416,5.885293,0.757034,1.579765,1.847644,0.0,5.342715,NaN
min,20.000000,0.000000,1.000000,1.000000,1.000000,1.000000,27.474722,-100.214723,1085.0,1.000000,NaN
25%,448.250000,1015.000000,1.000000,1.000000,23.000000,7.000000,29.005356,-98.632083,1085.0,1.000000,NaN
50%,753.500000,1207.000000,1.000000,1.000000,23.000000,7.000000,30.628972,-96.906667,1085.0,1.000000,NaN
75%,1350.000000,1425.000000,1.000000,1.000000,23.000000,7.000000,31.941667,-95.344445,1085.0,2.000000,NaN
max,6211.000000,1845.000000,2.000000,16.000000,24.000000,14.000000,33.443611,-93.776667,1085.0,42.200000,NaN


In [48]:
# arsenic_df = pre_process(arsenic,[0,2,6,10e10],["zero_arsenic","moderate_arsenic",'high_arsenic'])
# copper_df = pre_process(copper,[0,2.5,10,10e10],["zero_copper","moderate_copper",'high_copper'])
# sulfate_df = pre_process(sulfate,[0,40,300,10e10],["zero_sulfate","moderate_sulfate",'high_sulfate'])
# boran_df = pre_process(boran,[0,300,1500,10e10],["zero_boran","moderate_boran",'high_boran'])
# chloride_df = pre_process(chloride,[0,80,400,10e10],["zero_chloride","moderate_chloride",'high_chloride'])
# molybdenum_df = pre_process(molybdenum,[0,2,10,10e10],["zero_molybdenum","moderate_molybdenum",'high_molybdenum'])
# vanadium_df = pre_process(vanadium,[0,2.5,10,10e10],["zero_vanadium","moderate_vanadium",'high_vanadium'])
arsenic_df = pre_process(arsenic,["arsenic"],thres = 2.75/2)
copper_df = pre_process(copper,["copper"],thres = 4.91/2)
sulfate_df = pre_process(sulfate,["sulfate"],thres = 60.34/2)
boran_df = pre_process(boran,["boran"],thres = 418.79/2)
chloride_df = pre_process(chloride,["chloride"],thres = 96.95/2)
molybdenum_df = pre_process(molybdenum,["molybdenum"],thres = 2.87/2)
vanadium_df = pre_process(vanadium,["vanadium"], thres = 3.12/2)
# arsenic_df = pre_process(arsenic,["arsenic"],thres = 0)
# copper_df = pre_process(copper,["copper"],thres = 0)
# sulfate_df = pre_process(sulfate,["sulfate"],thres = 0)
# boran_df = pre_process(boran,["boran"],thres = 0)
# chloride_df = pre_process(chloride,["chloride"],thres = 0)
# molybdenum_df = pre_process(molybdenum,["molybdenum"],thres = 0)
# vanadium_df = pre_process(vanadium,["vanadium"], thres = 0)

In [5]:
arsenic_df.head()

,StateWellNumber,Date,LatitudeDD,LongitudeDD,ParameterValue,type
1329,5902805,19841221,30.888056,-96.816389,10.0,arsenic
389,3456704,19850513,32.147222,-95.104167,10.0,arsenic
1261,5855201,19880921,30.244444,-97.207222,10.0,arsenic
1290,5862303,19880927,30.096111,-97.251112,10.0,arsenic
1262,5855206,19880927,30.215834,-97.198611,10.0,arsenic


In [49]:
# typess = [["zero_arsenic","moderate_arsenic",'high_arsenic'],["zero_copper","moderate_copper",'high_copper'],\
#          ["zero_sulfate","moderate_sulfate",'high_sulfate'],["zero_boran","moderate_boran",'high_boran'],\
#          ["zero_chloride","moderate_chloride",'high_chloride'],["zero_molybdenum","moderate_molybdenum",'high_molybdenum'],\
#          ["zero_vanadium","moderate_vanadium",'high_vanadium']]
typess = ["arsenic","copper","sulfate","boran","chloride","molybdenum","vanadium"]
l = [arsenic_df,copper_df,sulfate_df,boran_df,chloride_df,molybdenum_df,vanadium_df]

In [50]:
df = get_dates(copper)
df['year'] =  df['Date']/10000
df['year'] = df['year'].astype('int')
df = df[df['year']==2002]
date1 = df['Date']

df = get_dates(vanadium)
df['year'] =  df['Date']/10000
df['year'] = df['year'].astype('int')
df = df[df['year']==2002]
date2 = df['Date']

dates = np.intersect1d(date1,date2)

In [51]:
sum_vec=[]
for i in range(len(l)):
    l[i] = vectorize(l[i],typess[i],dates=dates)
    if i==0:
        sum_vec = l[0]
    if i>=1:
        sum_vec = add_vectorization(sum_vec,l[i])

In [44]:
## todo 
## sum_vec is a list of dataframe with len 92
## i-th dataframe contains all the objects on date i
sum_vec[1]

,type,x,y
0,arsenic,-94.646667,31.386945
1,arsenic,-94.552501,31.779444
2,arsenic,-94.665001,31.763889
3,arsenic,-94.670556,31.438611
4,arsenic,-94.726944,31.398889
5,arsenic,-94.616112,31.718889
6,arsenic,-94.840000,31.815001
7,copper,-94.646667,31.386945
8,copper,-94.616112,31.718889
9,copper,-94.840000,31.815001


In [10]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

In [11]:
list(powerset([1,2,3]))

[(1,), (2,), (3,), (1, 2), (1, 3), (2, 3), (1, 2, 3)]

In [12]:
    colo = list(powerset(typess))[10]
    print(colo)
    PI = []
    for date in range(len(dates)):
        if not (all(x in list(sum_vec[date]['type']) for x in list(colo))):
            print(list(sum_vec[date]['type'].unique()))

('arsenic', 'chloride')
['arsenic', 'sulfate', 'boran', 'vanadium']
['arsenic', 'copper', 'sulfate', 'boran', 'vanadium']
['arsenic', 'sulfate', 'boran']
['arsenic', 'sulfate', 'boran']
['arsenic']
['arsenic', 'copper', 'sulfate']
['arsenic', 'copper', 'boran']
['arsenic', 'copper', 'sulfate']
['arsenic', 'copper', 'sulfate', 'boran']
['arsenic', 'copper', 'sulfate']
['arsenic', 'copper', 'sulfate', 'boran', 'vanadium']
['arsenic', 'sulfate', 'boran', 'molybdenum']
['arsenic']
['arsenic', 'copper', 'boran']
['arsenic', 'copper', 'sulfate', 'boran']
['arsenic', 'sulfate']
['arsenic', 'sulfate', 'boran']
['arsenic', 'sulfate']
['arsenic', 'sulfate']
['arsenic', 'boran']
['arsenic', 'sulfate', 'boran', 'vanadium']
['arsenic']
['arsenic', 'copper', 'vanadium']
['arsenic', 'copper', 'sulfate']
['arsenic', 'copper']


In [94]:
# first find colocation pattern
thres = 0.5
patterns = []
num_colos = dict()
diff_colos = dict()
for colo in list(powerset(typess)):
    PI = []
    num_colo = []
    for date in range(len(dates)):
        if not (all(x in list(sum_vec[date]['type']) for x in list(colo))):
            PI.append(0)
            num_colo.append(0)
            continue
        arr=sum_vec[date].to_numpy()
        dic=get_dict(arr,key_idx=0)
        colo_array=get_colocations(dic,colo_type=list(colo),thres=0.001)
        if colo_array is None: 
            num_colo.append(0)
        else: 
            num_colo.append(len(colo_array))
        PI.append(get_modified_part_index(dic,colo_type=list(colo),colo_arr=colo_array))
    if np.mean(PI)>=thres:
        #print(colo)
        patterns.append(colo)
        num_colos[str(colo)] = num_colo
        diff_colo = np.array(num_colo[1:])-np.array(num_colo[:-1])
        diff_colos[str(colo)] = diff_colo
patterns

/Users/suhong/Desktop/UROP_Data/UROP/groundwater/calculator.py:124: RuntimeWarning: invalid value encountered in sqrt
  dists=np.sqrt(-2*pos_t2.dot(pos_t1.T)+np.sum(pos_t2**2,axis=1,keepdims=True)+np.sum(pos_t1**2,axis=1))


[('arsenic',),
 ('copper',),
 ('sulfate',),
 ('boran',),
 ('chloride',),
 ('arsenic', 'copper'),
 ('arsenic', 'sulfate'),
 ('arsenic', 'boran'),
 ('arsenic', 'chloride'),
 ('copper', 'boran'),
 ('sulfate', 'boran'),
 ('boran', 'chloride'),
 ('arsenic', 'copper', 'boran'),
 ('arsenic', 'sulfate', 'boran'),
 ('arsenic', 'boran', 'chloride')]

In [100]:
guo_index = "('arsenic', 'boran', 'chloride')"
guo = np.reshape(num_colos[guo_index],(-1,1))
guo = guo[1:]-guo[:-1]
causes = []
for colo in patterns:
    colo = str(colo)
    if colo!=guo_index:
        yin = np.reshape(num_colos[colo],(-1,1))
        yin = yin[1:]-yin[:-1]
        df = np.concatenate((guo,yin),axis=1)
        lags = select_time_lags(yin)
        results = grangercausalitytests(df, maxlag=[lags])
        if (results[lags])[0]['ssr_ftest'][1]<=0.1:
            causes.append(colo)


Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.6917  , p=0.5999  , df_denom=78, df_num=4
ssr based chi2 test:   chi2=3.0860  , p=0.5435  , df=4
likelihood ratio test: chi2=3.0325  , p=0.5524  , df=4
parameter F test:         F=0.6917  , p=0.5999  , df_denom=78, df_num=4

Granger Causality
number of lags (no zero) 8
ssr based F test:         F=1.0150  , p=0.4336  , df_denom=66, df_num=8
ssr based chi2 test:   chi2=10.2116 , p=0.2505  , df=8
likelihood ratio test: chi2=9.6306  , p=0.2919  , df=8
parameter F test:         F=1.0150  , p=0.4336  , df_denom=66, df_num=8

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=2.6787  , p=0.0377  , df_denom=78, df_num=4
ssr based chi2 test:   chi2=11.9512 , p=0.0177  , df=4
likelihood ratio test: chi2=11.1985 , p=0.0244  , df=4
parameter F test:         F=2.6787  , p=0.0377  , df_denom=78, df_num=4

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=3.2654  , p=0.0157  , df_d

In [101]:
causes

["('sulfate',)",
 "('boran',)",
 "('chloride',)",
 "('arsenic', 'sulfate')",
 "('arsenic', 'boran')",
 "('arsenic', 'chloride')",
 "('sulfate', 'boran')",
 "('arsenic', 'sulfate', 'boran')"]

In [75]:
yin = np.reshape(num_colos["('arsenic', 'sulfate')"],(-1,1))
guo = np.reshape(num_colos["('arsenic',)"],(-1,1))
yin = yin[1:]-yin[:-1]
guo = guo[1:]-guo[:-1]
df = np.concatenate((guo,yin),axis=1)

In [76]:
lags = select_time_lags(yin)
results = grangercausalitytests(df, maxlag=[lags])
results


Granger Causality
number of lags (no zero) 4
ssr based F test:         F=3.0799  , p=0.0207  , df_denom=78, df_num=4
ssr based chi2 test:   chi2=13.7413 , p=0.0082  , df=4
likelihood ratio test: chi2=12.7584 , p=0.0125  , df=4
parameter F test:         F=3.0799  , p=0.0207  , df_denom=78, df_num=4


{4: ({'ssr_ftest': (3.0799496270486264, 0.020741919030264927, 78.0, 4),
   'ssr_chi2test': (13.74131372067849, 0.008168129550053393, 4),
   'lrtest': (12.758364018660586, 0.012518856946197922, 4),
   'params_ftest': (3.079949627048634, 0.020741919030264785, 78.0, 4.0)},
   array([[0., 0., 0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 1., 0.]])])}

In [67]:
#get p-value of granger causality test
(results[4])[0]['ssr_ftest'][1]

0.004155445975629328

In [55]:
grangercausalitytests(df, maxlag=10)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0535  , p=0.8176  , df_denom=87, df_num=1
ssr based chi2 test:   chi2=0.0554  , p=0.8140  , df=1
likelihood ratio test: chi2=0.0553  , p=0.8140  , df=1
parameter F test:         F=0.0535  , p=0.8176  , df_denom=87, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.2866  , p=0.7516  , df_denom=84, df_num=2
ssr based chi2 test:   chi2=0.6072  , p=0.7381  , df=2
likelihood ratio test: chi2=0.6052  , p=0.7389  , df=2
parameter F test:         F=0.2866  , p=0.7516  , df_denom=84, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.3952  , p=0.7568  , df_denom=81, df_num=3
ssr based chi2 test:   chi2=1.2882  , p=0.7319  , df=3
likelihood ratio test: chi2=1.2788  , p=0.7342  , df=3
parameter F test:         F=0.3952  , p=0.7568  , df_denom=81, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.2158  , p=0.9289  , df_d

{1: ({'ssr_ftest': (0.05350595252279672, 0.8176143835858538, 87.0, 1),
   'ssr_chi2test': (0.0553509853684104, 0.8140008070272722, 1),
   'lrtest': (0.05533397161281073, 0.8140288719265617, 1),
   'params_ftest': (0.053505952522780134, 0.8176143835858861, 87.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (0.28656299479098546, 0.7515708324434764, 84.0, 2),
   'ssr_chi2test': (0.607240631818993, 0.7381410736743339, 2),
   'lrtest': (0.6051784269982932, 0.7389025652350032, 2),
   'params_ftest': (0.286562994790993, 0.751570832443473, 84.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (0.3952327220421422, 0.7567665173112876, 81.0, 3),
   'ssr_chi2test': (1.2881659088780932, 0.7319450986083647, 3),
   'lrtest': (1.2788286724706381, 0.7341664514291393, 3),
   'params_ftest': (0.39523272204214505, 0.7567665173112876, 81.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0.,

In [30]:
pacf = statsmodels.tsa.stattools.pacf(yin)
pacf

array([ 1.        , -0.26634513, -0.41448164, -0.27876701, -0.32812782,
       -0.07695989, -0.16502999, -0.13954905, -0.3058271 ,  0.08497297,
        0.14815018,  0.15747823, -0.29483484, -0.06697657, -0.04206294,
        0.12351361, -0.19783228, -0.06218909,  0.22208825, -0.07666888])

In [20]:
thres = 0.1
for i in range(len(pacf)):
    if abs(pacf[i])<=thres:
        return i+1

SyntaxError: 'return' outside function (162891836.py, line 4)

In [ ]:
x = np.array(num_colos["('sulfate',)"])
y = np.array(num_colos["('arsenic', 'boran', 'chloride')"])
z = np.array(num_colos["('arsenic', 'sulfate')"])
x = x[1:]-x[:-1]
y = y[1:]-y[:-1]
z = z[1:]-z[:-1]
dic = {'x':x,'y':y,'z':z} 
cov_df = pd.DataFrame(dic,columns = ['x','y','z'])
# test x y z and z y x
pg.partial_corr(data=cov_df, x='z', y='y', covar='x')

In [ ]:
patterns

In [ ]:
X = np.reshape(num_colos["('sulfate',)"],(-1,1))
y = np.reshape(num_colos["('arsenic',)"],(-1,1))
lm = linear_model.LinearRegression()
model = lm.fit(X,y)

In [ ]:
lm.score(y,X)

In [ ]:
select_time_lags(yin)

In [95]:
df = pd.DataFrame(diff_colos)

In [96]:
df.to_csv("Data/diff_colos.csv")